In [13]:
import codecs
import glob
import pandas as pd


In [14]:
data_dir = "data/input/test/*.out"
data_names = data_dir.split('/')[2:-1]
data_name = ""
for p in data_names:
    data_name = data_name + p
result_dir = "data/output/" + data_name + ".csv"
paths = [p for p in glob.glob(data_dir)]
cvd_key = "Threshold  Converged?"
SCF_done_key = "SCF Done"
energy_list = []
not_cvd_list = []
cvd_list = []
df_output = pd.DataFrame()

In [15]:
for path in paths:
    # file_nameから角度の情報を取る
    file_name = path.split('/')[-1]
    torsion_angle = int(file_name.split("@")[0].split("_")[-1])
    with codecs.open(path, "r", "UTF-8", "ignore") as file:
        df = pd.read_table(file)
        df = df.rename(columns={df.columns[0]: 'Col_1'})
        
#         converged: YESを探す
        cvd_key_df = df[df['Col_1'].str.contains(cvd_key, case=False)]
        idx_list = list(cvd_key_df.index)
        for idx in idx_list:
            if list(df.iloc[idx+1:idx+5]['Col_1'].str.contains("YES", case=False)) == [True, True, True, True]:
                df_tgt = df[idx_list[idx_list.index(idx)-1]:idx]
                energy = float(df_tgt[df_tgt['Col_1'].str.contains("SCF Done")].iloc[0][0].split('=')[-1].split("A.U.")[0].replace(" ",""))
                df_E = pd.DataFrame(data=[energy], 
                            columns=[torsion_angle],
                            index=["SCF done"]).T
                df_output = pd.concat([df_output, df_E])
                cvd_list.append(torsion_angle)
            

df_output = df_output.sort_index()
df_output.to_csv(result_dir)
df_output

,SCF done
0,-1.012728e+03
2,-1.012000e+11
